# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [3]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [4]:
show(boston, allcols=true)

506×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 0.00632 │ 18.0    │ 2.31    │ 0     │ 0.538   │ 6.575   │ 65.2    │
│ 2   │ 0.02731 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 6.421   │ 78.9    │
│ 3   │ 0.02729 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 7.185   │ 61.1    │
│ 4   │ 0.03237 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.998   │ 45.8    │
│ 5   │ 0.06905 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 7.147   │ 54.2    │
│ 6   │ 0.02985 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.43    │ 58.7    │
│ 7   │ 0.08829 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.012   │ 66.6    │
│ 8   │ 0.14455 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.172   │ 96.1    │
│ 9   │ 0.21124 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 5.631   │ 100.0   │
│ 10  │ 0.17004 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.004   │ 85.9 

In [5]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))

506-element view(::Array{Int64,1}, [466, 158, 192, 22, 412, 405, 16, 452, 115, 414  …  391, 373, 257, 483, 370, 104, 23, 189, 90, 238]) with eltype Int64:
 466
 158
 192
  22
 412
 405
  16
 452
 115
 414
 411
 276
 351
   ⋮
  84
 234
 391
 373
 257
 483
 370
 104
  23
 189
  90
 238

In [8]:
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [13]:
train = boston[train_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,3.1636,0.0,18.1,0,0.655,5.759,48.2,3.0665,24,666
2,1.22358,0.0,19.58,0,0.605,6.943,97.4,1.8773,5,403
3,0.06911,45.0,3.44,0,0.437,6.739,30.8,6.4798,5,398
4,0.85204,0.0,8.14,0,0.538,5.965,89.2,4.0123,4,307
5,14.0507,0.0,18.1,0,0.597,6.657,100.0,1.5275,24,666
6,41.5292,0.0,18.1,0,0.693,5.531,85.4,1.6074,24,666
7,0.62739,0.0,8.14,0,0.538,5.834,56.5,4.4986,4,307
8,5.44114,0.0,18.1,0,0.713,6.655,98.2,2.3552,24,666
9,0.14231,0.0,10.01,0,0.547,6.254,84.2,2.2565,6,432


In [15]:
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,1.20742,0.0,19.58,0,0.605,5.875,94.6,2.4259,5,403
2,4.34879,0.0,18.1,0,0.58,6.167,84.0,3.0334,24,666
3,0.04527,0.0,11.93,0,0.573,6.12,76.7,2.2875,1,273
4,0.06664,0.0,4.05,0,0.51,6.546,33.1,3.1323,5,296
5,11.8123,0.0,18.1,0,0.718,6.824,76.5,1.794,24,666
6,0.79041,0.0,9.9,0,0.544,6.122,52.8,2.6403,4,304
7,0.18836,0.0,6.91,0,0.448,5.786,33.3,5.1004,3,233
8,0.31533,0.0,6.2,0,0.504,8.266,78.3,2.8944,8,307
9,6.28807,0.0,18.1,0,0.74,6.341,96.4,2.072,24,666


In [19]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)     Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)   25.5452      5.51608      4.63    <1e-5    14.7003      36.3901
Crim          -0.101817    0.0343544   -2.96    0.0032   -0.169359    -0.0342742
Zn             0.0353702   0.0139913    2.53    0.0119    0.00786257   0.0628779
Indus          0.05444     0.0673852    0.81    0.4196   -0.0780427    0.186923
Chas           1.81631     0.882464     2.06    0.0402    0.0813418    3.55128
NOx          -16.6164      3.89164     -4.27    <1e-4   -24.2676      -8.

In [20]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 22.32552986074361
 19.34855360498944
 21.926816810238847
 30.333600661904825
 16.08861471365867
 25.92667002806424
 21.22478543984195
 39.26987589623995
 17.22505874585954
 24.518048255156643
 35.472376382553286
 21.05080354439182
 17.942332803933304
  ⋮
 24.504748987785426
 38.27023552549985
 15.645789254290957
 22.192828993071558
 37.60656733838073
 27.404052249785067
 28.872564306742355
 20.038093302170225
 16.886794505300497
 31.496417640798647
 31.039651142193396
 32.78317793759356

In [21]:
r2(ols)

0.7771571608562774

In [22]:
adjr2(ols)

0.7697480639026498